# Package Load

In [1]:
import preprocessing
import tensorflow as tf
import numpy as np
import random
import sys
import utils
import pickle

# preprocessing : 이미지 파일로 존재하는 데이터를 numpy와 onehotencoding 라벨로 바꿔주는 모듈
# tensorflow : 텐서플로우 , 일반적으로 tf로 줄여서 사용
# numpy : fundamental package for scientific computing with Python
# pickle : python 의 데이터 형식을 그대로 저장하기 위한 파이썬 기본 패키지


# Image Loading

In [2]:
train_imgs , train_labs , test_imgs , test_labs=preprocessing.get_cifar_data()

def next_batch(imgs , labs , batch_size):
    indices=random.sample(range(len(imgs)) , batch_size)
    batch_xs=imgs[indices]
    for x,index in zip(batch_xs,range(len(batch_xs))) :
        
        r = random.random()
        if r > 0.5 :
            batch_xs[index] = np.fliplr(x)
    batch_ys=labs[indices]
    return batch_xs , batch_ys

# preprocessing 모듈에서 train_imgs , train_labs , test_imgs , test_labs 를 불러 온다.
# fully connected 모델을 위해 32x32x3 크기의 이미지를 3072 으로 reshape 한다.
# next_batch 함수는 batch_size 수 만큼 random 으로 추출하여 train batch 를 생성하는 함수


# Set Placeholder And Variables

In [3]:

height = 32
width = 32
color_ch=3
n_classes = 10
learning_rate=0.001
max_iter=10000
check_point=100
x_ =tf.placeholder(tf.float32, [ None , height , width , color_ch ])
y_ =tf.placeholder( tf.int32 , [ None , n_classes ])

p_keep_conv = tf.placeholder(tf.float32,name="p_keep_conv")

# Placeholder 는 차후에 입력할 값
# Variables 는 weight 로 학습시 변하는 값


# Network Modeling

In [4]:
out_ch=28
w1=tf.get_variable("w1" , [7,7,color_ch , out_ch] , initializer=tf.contrib.layers.xavier_initializer())
b1=tf.Variable(tf.constant(0.1) ,out_ch)
s1=[1,1,1,1]
p1='SAME'
layer1=tf.nn.conv2d(x_ , w1 , s1 , p1 )+b1
layer1=tf.nn.relu(layer1)


out_ch2=64
w2=tf.get_variable("w2" , [5,5,out_ch, out_ch2] , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.constant(0.1) ,out_ch2)
s2=[1,1,1,1]
layer2=tf.nn.conv2d(layer1, w2 , s2, padding='SAME')+b2
layer2=tf.nn.relu(layer2)


pool_s = [1,2,2,1]
layer3 = tf.nn.max_pool(layer2, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')
layer3 = tf.nn.dropout(layer3, p_keep_conv)


out_ch3=128
w3=tf.get_variable("w3" , [5,5,out_ch2, out_ch3] , initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.constant(0.1) ,out_ch3)
s3=[1,1,1,1]
layer4=tf.nn.conv2d(layer3, w3 , s3, padding='SAME')+b3
layer4=tf.nn.relu(layer4)

out_ch4=128
w4=tf.get_variable("w4" , [5,5,out_ch3, out_ch4] , initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.constant(0.1) ,out_ch4)
s4=[1,1,1,1]
layer4=tf.nn.conv2d(layer4, w4 , s4, padding='SAME')+b4
layer4=tf.nn.relu(layer4)

layer5 = tf.nn.max_pool(layer4, ksize=[1, 2, 2, 1], strides=pool_s, padding='SAME')
layer5 = tf.nn.dropout(layer5, p_keep_conv)

#fully connected layer = affine layer

end_conv_layer=layer5
flatten_layer=tf.contrib.layers.flatten(end_conv_layer)
length=flatten_layer.get_shape()[1]
fc_w1=tf.get_variable("fc_w1" ,[length,n_classes])
fc_b1=tf.Variable(tf.constant(0.1) , n_classes)
y_conv=tf.matmul(flatten_layer ,fc_w1 )+fc_b1

# 4개의 convolution neural network 와 1개의 fully connected_layer 로 구성
# 2개의 convolution layer 를 거친 후 각 각 max pooling 적용 / max pooling 후에는 activation map 의 가로 세로 크기가 절반이 된다.
# stride : 좌우로 몇 칸 씩 커널을 이동 할 것인지에 대한 값
# padding : convolution 전후로 activation map 의 크기를 조정하기 위한 값 , SAME 을 입력하면 항상 convolution 전후의 크기가 같다.
# weight : [커널의 가로크기, 커널의 세로크기, input activation map 의 채널 크기, output activiation map 의 채널 크기]


# Define Functions

In [5]:
pred=tf.nn.softmax(y_conv)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= pred , labels=y_) , name='cost')
train=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost,name='train')
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y_, 1), name='correct_pred')
accuracy = tf.reduce_mean(tf.cast(correct_pred, dtype=tf.float32), name='accuracy')

# softmax : 결과값을 각 class 에 해당하는 확률값을로 리턴
# softmax_cross_entropy_with_logits : 실제 정답인 onehot vector 와 예측값 pred 를 차이를 cross_entropy 로 계산
# tf.train.GradientDescentOptimizer : cost 가 최소와 되도록 weight를 조정하는 함수
# accuracy : 실제 값과 예측값의 일치률

# Training

In [6]:
sess=tf.Session()
init=tf.global_variables_initializer()
sess.run(init)
saver = tf.train.Saver()

batch_iteration = 100    
training_epochs = 2000

train_cost_list = []
test_cost_list = []
train_acc_list = []
test_acc_list = []

# Tensorflow 학습을 위한 session 정의
# saver : 학습한 weight 와 graph 를 저장

# sess.run 을 통해 함수를 실행 , feed_dict 는 placeholder 에 들어갈 값


for epoch in range(training_epochs):
    
    train_avg_cost = 0.
    train_avg_acc = 0.
    test_avg_cost = 0.
    test_avg_acc = 0.
    
    for batch in range(batch_iteration) :
        batch_xs , batch_ys = next_batch(train_imgs , train_labs , 60)
        
        sess.run(train, feed_dict={x_:batch_xs , y_ : batch_ys,p_keep_conv:0.7})
        train_avg_cost += sess.run(cost, feed_dict={x_:batch_xs , y_ : batch_ys,p_keep_conv:1.0})
        train_avg_acc += sess.run(accuracy, feed_dict={x_:batch_xs , y_ : batch_ys,p_keep_conv:1.0})
        
    train_avg_cost = train_avg_cost / batch_iteration
    train_avg_acc = train_avg_acc / batch_iteration
    
    test_avg_cost = sess.run(cost, feed_dict={x_: test_imgs, y_: test_labs,p_keep_conv:1.0})
    test_avg_acc = sess.run(accuracy, feed_dict={x_: test_imgs, y_: test_labs,p_keep_conv:1.0})
    
    
    print "##################################"
    print ("Epoch: %03d/%03d train cost: %.4f" % (epoch, training_epochs, train_avg_cost))
    print ("Epoch: %03d/%03d train acc: %.4f" % (epoch, training_epochs, train_avg_acc))
    print ("Epoch: %03d/%03d test cost: %.4f" % (epoch, training_epochs, test_avg_cost))
    print ("Epoch: %03d/%03d test acc: %.4f" % (epoch, training_epochs, test_avg_acc))    
    
    train_cost_list.append(train_avg_cost)
    test_cost_list.append(test_avg_cost)
    train_acc_list.append(train_avg_acc)
    test_acc_list.append(test_avg_acc)
   
    
    
saver.save(sess, "model/cifar_deep_convolution-block-overfitting.ckpt")

with open('cost_acc/cifar_deep_convolution-block-overfitting_train_cost', 'wb') as fp:
    pickle.dump(train_cost_list, fp)
with open('cost_acc/cifar_deep_convolution-block-overfitting_test_cost', 'wb') as fp:
    pickle.dump(test_cost_list, fp)
with open('cost_acc/cifar_deep_convolution-block-overfitting_train_acc', 'wb') as fp:
    pickle.dump(train_acc_list, fp)
with open('cost_acc/cifar_deep_convolution-block-overfitting_test_acc', 'wb') as fp:
    pickle.dump(test_acc_list, fp)
    

##################################
Epoch: 000/2000 train cost: 2.3036
Epoch: 000/2000 train acc: 0.0967
Epoch: 000/2000 test cost: 2.3030
Epoch: 000/2000 test acc: 0.1016
##################################
Epoch: 001/2000 train cost: 2.3031
Epoch: 001/2000 train acc: 0.1005
Epoch: 001/2000 test cost: 2.3028
Epoch: 001/2000 test acc: 0.1016
##################################
Epoch: 002/2000 train cost: 2.3030
Epoch: 002/2000 train acc: 0.0993
Epoch: 002/2000 test cost: 2.3026
Epoch: 002/2000 test acc: 0.1016
##################################
Epoch: 003/2000 train cost: 2.3024
Epoch: 003/2000 train acc: 0.0978
Epoch: 003/2000 test cost: 2.3025
Epoch: 003/2000 test acc: 0.1016
##################################
Epoch: 004/2000 train cost: 2.3022
Epoch: 004/2000 train acc: 0.1078
Epoch: 004/2000 test cost: 2.3023
Epoch: 004/2000 test acc: 0.1153
##################################
Epoch: 005/2000 train cost: 2.3025
Epoch: 005/2000 train acc: 0.1223
Epoch: 005/2000 test cost: 2.3022
Epoch: 

##################################
Epoch: 048/2000 train cost: 2.2696
Epoch: 048/2000 train acc: 0.1858
Epoch: 048/2000 test cost: 2.2658
Epoch: 048/2000 test acc: 0.2011
##################################
Epoch: 049/2000 train cost: 2.2652
Epoch: 049/2000 train acc: 0.1985
Epoch: 049/2000 test cost: 2.2631
Epoch: 049/2000 test acc: 0.2002
##################################
Epoch: 050/2000 train cost: 2.2614
Epoch: 050/2000 train acc: 0.1977
Epoch: 050/2000 test cost: 2.2603
Epoch: 050/2000 test acc: 0.1954
##################################
Epoch: 051/2000 train cost: 2.2564
Epoch: 051/2000 train acc: 0.1972
Epoch: 051/2000 test cost: 2.2578
Epoch: 051/2000 test acc: 0.1828
##################################
Epoch: 052/2000 train cost: 2.2584
Epoch: 052/2000 train acc: 0.1790
Epoch: 052/2000 test cost: 2.2556
Epoch: 052/2000 test acc: 0.1869
##################################
Epoch: 053/2000 train cost: 2.2536
Epoch: 053/2000 train acc: 0.1958
Epoch: 053/2000 test cost: 2.2541
Epoch: 

##################################
Epoch: 096/2000 train cost: 2.1864
Epoch: 096/2000 train acc: 0.2650
Epoch: 096/2000 test cost: 2.1854
Epoch: 096/2000 test acc: 0.2673
##################################
Epoch: 097/2000 train cost: 2.1808
Epoch: 097/2000 train acc: 0.2727
Epoch: 097/2000 test cost: 2.1859
Epoch: 097/2000 test acc: 0.2704
##################################
Epoch: 098/2000 train cost: 2.1863
Epoch: 098/2000 train acc: 0.2625
Epoch: 098/2000 test cost: 2.1834
Epoch: 098/2000 test acc: 0.2715
##################################
Epoch: 099/2000 train cost: 2.1794
Epoch: 099/2000 train acc: 0.2712
Epoch: 099/2000 test cost: 2.1839
Epoch: 099/2000 test acc: 0.2727
##################################
Epoch: 100/2000 train cost: 2.1767
Epoch: 100/2000 train acc: 0.2757
Epoch: 100/2000 test cost: 2.1839
Epoch: 100/2000 test acc: 0.2684
##################################
Epoch: 101/2000 train cost: 2.1812
Epoch: 101/2000 train acc: 0.2690
Epoch: 101/2000 test cost: 2.1812
Epoch: 

##################################
Epoch: 144/2000 train cost: 2.1512
Epoch: 144/2000 train acc: 0.3023
Epoch: 144/2000 test cost: 2.1516
Epoch: 144/2000 test acc: 0.3048
##################################
Epoch: 145/2000 train cost: 2.1427
Epoch: 145/2000 train acc: 0.3155
Epoch: 145/2000 test cost: 2.1513
Epoch: 145/2000 test acc: 0.3029
##################################
Epoch: 146/2000 train cost: 2.1435
Epoch: 146/2000 train acc: 0.3150
Epoch: 146/2000 test cost: 2.1519
Epoch: 146/2000 test acc: 0.3005
##################################
Epoch: 147/2000 train cost: 2.1470
Epoch: 147/2000 train acc: 0.3038
Epoch: 147/2000 test cost: 2.1499
Epoch: 147/2000 test acc: 0.3059
##################################
Epoch: 148/2000 train cost: 2.1461
Epoch: 148/2000 train acc: 0.3108
Epoch: 148/2000 test cost: 2.1498
Epoch: 148/2000 test acc: 0.3045
##################################
Epoch: 149/2000 train cost: 2.1503
Epoch: 149/2000 train acc: 0.3020
Epoch: 149/2000 test cost: 2.1510
Epoch: 

##################################
Epoch: 192/2000 train cost: 2.1215
Epoch: 192/2000 train acc: 0.3395
Epoch: 192/2000 test cost: 2.1272
Epoch: 192/2000 test acc: 0.3309
##################################
Epoch: 193/2000 train cost: 2.1156
Epoch: 193/2000 train acc: 0.3460
Epoch: 193/2000 test cost: 2.1290
Epoch: 193/2000 test acc: 0.3272
##################################
Epoch: 194/2000 train cost: 2.1211
Epoch: 194/2000 train acc: 0.3402
Epoch: 194/2000 test cost: 2.1297
Epoch: 194/2000 test acc: 0.3282
##################################
Epoch: 195/2000 train cost: 2.1170
Epoch: 195/2000 train acc: 0.3440
Epoch: 195/2000 test cost: 2.1266
Epoch: 195/2000 test acc: 0.3300
##################################
Epoch: 196/2000 train cost: 2.1145
Epoch: 196/2000 train acc: 0.3478
Epoch: 196/2000 test cost: 2.1261
Epoch: 196/2000 test acc: 0.3305
##################################
Epoch: 197/2000 train cost: 2.1158
Epoch: 197/2000 train acc: 0.3478
Epoch: 197/2000 test cost: 2.1271
Epoch: 

...

##################################
Epoch: 1807/2000 train cost: 1.8637
Epoch: 1807/2000 train acc: 0.6055
Epoch: 1807/2000 test cost: 1.9003
Epoch: 1807/2000 test acc: 0.5596
##################################
Epoch: 1808/2000 train cost: 1.8676
Epoch: 1808/2000 train acc: 0.5997
Epoch: 1808/2000 test cost: 1.9017
Epoch: 1808/2000 test acc: 0.5579
##################################
Epoch: 1809/2000 train cost: 1.8648
Epoch: 1809/2000 train acc: 0.6010
Epoch: 1809/2000 test cost: 1.9017
Epoch: 1809/2000 test acc: 0.5584
##################################
Epoch: 1810/2000 train cost: 1.8705
Epoch: 1810/2000 train acc: 0.5977
Epoch: 1810/2000 test cost: 1.9027
Epoch: 1810/2000 test acc: 0.5569
##################################
Epoch: 1811/2000 train cost: 1.8690
Epoch: 1811/2000 train acc: 0.5983
Epoch: 1811/2000 test cost: 1.9011
Epoch: 1811/2000 test acc: 0.5613
##################################
Epoch: 1812/2000 train cost: 1.8660
Epoch: 1812/2000 train acc: 0.6023
Epoch: 1812/2000 te

##################################
Epoch: 1854/2000 train cost: 1.8708
Epoch: 1854/2000 train acc: 0.5942
Epoch: 1854/2000 test cost: 1.8962
Epoch: 1854/2000 test acc: 0.5663
##################################
Epoch: 1855/2000 train cost: 1.8546
Epoch: 1855/2000 train acc: 0.6123
Epoch: 1855/2000 test cost: 1.8985
Epoch: 1855/2000 test acc: 0.5625
##################################
Epoch: 1856/2000 train cost: 1.8642
Epoch: 1856/2000 train acc: 0.6027
Epoch: 1856/2000 test cost: 1.8965
Epoch: 1856/2000 test acc: 0.5652
##################################
Epoch: 1857/2000 train cost: 1.8673
Epoch: 1857/2000 train acc: 0.5980
Epoch: 1857/2000 test cost: 1.8972
Epoch: 1857/2000 test acc: 0.5629
##################################
Epoch: 1858/2000 train cost: 1.8570
Epoch: 1858/2000 train acc: 0.6102
Epoch: 1858/2000 test cost: 1.8944
Epoch: 1858/2000 test acc: 0.5654
##################################
Epoch: 1859/2000 train cost: 1.8728
Epoch: 1859/2000 train acc: 0.5923
Epoch: 1859/2000 te

##################################
Epoch: 1901/2000 train cost: 1.8566
Epoch: 1901/2000 train acc: 0.6102
Epoch: 1901/2000 test cost: 1.8905
Epoch: 1901/2000 test acc: 0.5703
##################################
Epoch: 1902/2000 train cost: 1.8603
Epoch: 1902/2000 train acc: 0.6058
Epoch: 1902/2000 test cost: 1.8952
Epoch: 1902/2000 test acc: 0.5656
##################################
Epoch: 1903/2000 train cost: 1.8618
Epoch: 1903/2000 train acc: 0.6037
Epoch: 1903/2000 test cost: 1.8916
Epoch: 1903/2000 test acc: 0.5693
##################################
Epoch: 1904/2000 train cost: 1.8643
Epoch: 1904/2000 train acc: 0.6048
Epoch: 1904/2000 test cost: 1.8917
Epoch: 1904/2000 test acc: 0.5660
##################################
Epoch: 1905/2000 train cost: 1.8534
Epoch: 1905/2000 train acc: 0.6130
Epoch: 1905/2000 test cost: 1.8914
Epoch: 1905/2000 test acc: 0.5699
##################################
Epoch: 1906/2000 train cost: 1.8585
Epoch: 1906/2000 train acc: 0.6043
Epoch: 1906/2000 te

##################################
Epoch: 1948/2000 train cost: 1.8452
Epoch: 1948/2000 train acc: 0.6217
Epoch: 1948/2000 test cost: 1.8874
Epoch: 1948/2000 test acc: 0.5735
##################################
Epoch: 1949/2000 train cost: 1.8502
Epoch: 1949/2000 train acc: 0.6143
Epoch: 1949/2000 test cost: 1.8866
Epoch: 1949/2000 test acc: 0.5727
##################################
Epoch: 1950/2000 train cost: 1.8631
Epoch: 1950/2000 train acc: 0.6047
Epoch: 1950/2000 test cost: 1.8880
Epoch: 1950/2000 test acc: 0.5737
##################################
Epoch: 1951/2000 train cost: 1.8505
Epoch: 1951/2000 train acc: 0.6173
Epoch: 1951/2000 test cost: 1.8887
Epoch: 1951/2000 test acc: 0.5716
##################################
Epoch: 1952/2000 train cost: 1.8480
Epoch: 1952/2000 train acc: 0.6197
Epoch: 1952/2000 test cost: 1.8888
Epoch: 1952/2000 test acc: 0.5706
##################################
Epoch: 1953/2000 train cost: 1.8512
Epoch: 1953/2000 train acc: 0.6147
Epoch: 1953/2000 te

##################################
Epoch: 1995/2000 train cost: 1.8468
Epoch: 1995/2000 train acc: 0.6212
Epoch: 1995/2000 test cost: 1.8825
Epoch: 1995/2000 test acc: 0.5773
##################################
Epoch: 1996/2000 train cost: 1.8413
Epoch: 1996/2000 train acc: 0.6270
Epoch: 1996/2000 test cost: 1.8825
Epoch: 1996/2000 test acc: 0.5771
##################################
Epoch: 1997/2000 train cost: 1.8434
Epoch: 1997/2000 train acc: 0.6255
Epoch: 1997/2000 test cost: 1.8863
Epoch: 1997/2000 test acc: 0.5749
##################################
Epoch: 1998/2000 train cost: 1.8408
Epoch: 1998/2000 train acc: 0.6285
Epoch: 1998/2000 test cost: 1.8842
Epoch: 1998/2000 test acc: 0.5781
##################################
Epoch: 1999/2000 train cost: 1.8543
Epoch: 1999/2000 train acc: 0.6118
Epoch: 1999/2000 test cost: 1.8834
Epoch: 1999/2000 test acc: 0.5735
